In [19]:
import pandas as pd
import sqlite3 as sq
import numpy as np

In [45]:
online_store=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2.csv", delimiter=";")
orders=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Order-Order.csv", delimiter=";")
customers=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Customer-Customer.csv",delimiter=";")
products=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Product-Table 1.csv",delimiter=";")
sales=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Sales-Sales.csv",delimiter=";")
shipment=pd.read_csv("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/Sample-Superstore2/Shipment-Shipment.csv",delimiter=";")

FULL TABLE FOR EDA 

In [46]:
online_store.drop(labels="Row ID",axis=1,inplace=True)

In [47]:
online_store.rename(columns={"Order ID":"Order_ID","Order Date":"Order_Date","Ship Date":"Ship_Date","Ship Mode":"Ship_Mode","Customer ID":"Customer_ID","Customer Name":"Customer_Name","Postal Code":"Postal_Code","Product ID":"Product_ID","Sub-Category":"Sub_Category","Product Name":"Product_Name"},inplace=True)

In [48]:
online_store=pd.merge(online_store,products2,how="left",on=["Product_ID","Product_Name"])

In [49]:
online_store.drop(labels="Product_ID",axis=1,inplace=True)


REPLACING WRONG Product_ID

In [50]:
sales=pd.merge(sales,products,how="left",on=["Product_ID","Product_Name"])

In [51]:
sales.drop(labels=["Product_ID",'Sub_Category','Category','Product_Name'],axis=1,inplace=True)
products.drop(labels=["Product_ID"],axis=1,inplace=True)


In [52]:

sales.rename(columns={"Product_ID_NEW":"Product_ID"},inplace=True)
products.rename(columns={"Product_ID_NEW":"Product_ID"},inplace=True)

In [29]:
#orders = orders.assign(id=range(1,1+len(orders)))

In [53]:
sales=sales.iloc[:,[0,1,6,2,3,4,5]]
products=products.iloc[:,[3,0,1,2]]

In [32]:
#orders.to_csv('/Users/Yanaorfin/Documents/Academy/E-Commerce/Order-Order fixed.csv',index=False)

CREATING DATA BASE

In [33]:
db_conn = sq.connect("/Users/Yanaorfin_1/Documents/Academy/E_Commerce/onlinestore3.db")

In [34]:
# creating cursor object
cursor=db_conn.cursor()

In [36]:
# CUSTOMERS 

cursor.execute(
    """
    CREATE TABLE customers (
        Customer_ID VARCHAR (60) NOT NULL PRIMARY KEY, 
        Customer_Name VARCHAR (100) NOT NULL,
        Segment VARCHAR (100)
    );
    """
)



In [37]:
# PRODUCTS 

cursor.execute(
    """
    CREATE TABLE products (
        Product_id VARCHAR (60) NOT NULL PRIMARY KEY,
        Category VARCHAR (100)VARCHAR (100)VARCHAR (100),
        Sub_Category VARCHAR (100),
        Product_Name VARCHAR (200)
    );
    """
)

# ORDERS 
cursor.execute(
    """
    CREATE TABLE orders (
        Order_Id VARCHAR (60) PRIMARY KEY NOT NULL,
        Order_Date VARCHAR (20) NOT NULL,
        Customer_ID VARCHAR (60),
        FOREIGN KEY (Customer_ID) REFERENCES customers (Customer_ID)
    );
    """ 
)

#SHIPMENT

cursor.execute(
    """
    CREATE TABLE shipment (
        Shipment_id INTEGER PRIMARY KEY NOT NULL,
        Order_Id VARCHAR (60) NOT NULL,
        Ship_Date VARCHAR (20) NOT NULL, 
        Ship_Mode VARCHAR (60),
        Country VARCHAR (60),
        City VARCHAR (60),
        State VARCHAR (60),
        Postal_Code VARCHAR (60),
        Region VARCHAR (60),
        FOREIGN KEY (Order_Id) REFERENCES orders (Order_Id) 
    );
    """
)

# SALES 

cursor.execute(
    """
    CREATE TABLE sales (
        Sales_Id INTEGER PRIMARY KEY NOT NULL,
        Order_Id VARCHAR (60) NOT NULL,
        Product_ID VARCHAR (60),
        Sales REAL,
        Discount REAL,
        Profit REAL,
        Quantity INTEGER,
        FOREIGN KEY (Order_Id) REFERENCES orders (Order_Id),
        FOREIGN KEY (Product_ID) REFERENCES products (Product_ID)

);
"""
) 

In [ ]:

sales.to_sql('sales', db_conn, if_exists='append', index=False)



In [42]:
customers.to_sql('customers', db_conn, if_exists='append', index=False)

793

In [56]:
orders.to_sql('orders', db_conn, if_exists='append', index=False)

5009

In [40]:
products.to_sql('products', db_conn, if_exists='append', index=False)
shipment.to_sql('shipment',db_conn, if_exists='append', index=False)

5009

In [57]:
db_conn.close()